In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.ensemble_model-backuo as em 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer

In [4]:

df = pd.read_csv(r'/root/autodl-tmp/CommitFit/dataset/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df
df['command'] = df['diff'].apply(lambda x : ' '.join(x.split('@@')[:2]))
df
df_dataset = []
for index, row in df.iterrows():
  df_dataset.append([row['message'],row['command'],row['label']])

/tmp/ipykernel_1340/2618084632.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


In [5]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/bert-base-cased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')

Some weights of the model checkpoint at /root/autodl-tmp/models/bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [17]:
# Split the dataset
# preprocesser = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer)
train_data, val_data = train_test_split(df_dataset, test_size=0.2, random_state=1)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer)
val_dataset = preprocesser.SentencePairDataset(val_data, bert_tokenizer, codebert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)




# # Initialize the model
# model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# # Train the model
# model.trainer(train_loader, val_loader,num_epochs=10)



base_model1 = em.BaseModel(bert_model,bert_tokenizer)
base_model2 = em.BaseModel(codebert_model,codebert_tokenizer)

# Create stacking model
stacking_model = em.StackingModel(base_model1, base_model2)
# Train the model
stacking_model.trainer(train_loader, val_loader,num_epochs=12)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/12, Loss: 0.0990113877590558
Epoch 2/12, Loss: 0.05699514417877796
Epoch 3/12, Loss: 0.04168494194877157
Epoch 4/12, Loss: 0.030777447279372434
Epoch 5/12, Loss: 0.028709152421862047
Epoch 6/12, Loss: 0.022664402571981664
Epoch 7/12, Loss: 0.02724408796670241
Epoch 8/12, Loss: 0.026725432173752277
Epoch 9/12, Loss: 0.017468110373732703
Epoch 10/12, Loss: 0.02127898242984316
Epoch 11/12, Loss: 0.020804244124555702
Epoch 12/12, Loss: 0.02167102491943577
Validation Accuracy: 0.9668808699950568
Precision: 0.9528061224489796
Recall: 0.9613899613899614
F1-Score: 0.957078795643818
